In [76]:
import numpy as np 
import keras
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
import wave, os, glob
import warnings
import tflearn
import tensorflow as tf
import librosa
import h5py
from python_speech_features import mfcc
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Concatenate, concatenate, Conv2D
from keras.layers import Input, Dense, LSTM, Flatten, InputLayer
from keras import optimizers

In [77]:
import math

In [78]:
from tensorflow import keras
K = keras.backend

In [79]:
from keras.layers.normalization import BatchNormalization

In [80]:
# tflearn.init_graph(soft_placement=False)

In [81]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1618446486912853224, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2273558528
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3098510022847667842
 physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0, compute capability: 5.2"]

In [82]:
# tf.reset_default_graph()
tflearn.init_graph(num_cores=6,gpu_memory_fraction=1)

intra_op_parallelism_threads: 6
inter_op_parallelism_threads: 6
gpu_options {
  per_process_gpu_memory_fraction: 1
}
allow_soft_placement: true

In [83]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [88]:
def make_true(y):
    y_t = []
    for i in range(0, len(y)):
        y_t.append(sum(y[i]))
    return y_t

In [89]:
# custom loss function:
def test_cosine_loss_3(w, f):
    def test_cosine_loss_3(y_true, y_pred, batch_size = 64):
        print(np.shape(y_true), np.shape(y_pred))
        print(np.shape(w))
        var_weights = w
        normed_weights = tf.nn.l2_normalize(var_weights, (0,1), 1e-10, name='weights_norm')
        normed_features = tf.nn.l2_normalize(f, (0,1), 1e-10, name='features_norm')
        cosine = tf.matmul(normed_features, normed_weights)
        cosine = cosine - 0.15*y_true
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=50.*cosine))
        return K.sum(y_pred - y_pred, axis=-1) + loss
    print ("ok")
    return test_cosine_loss_3

In [194]:
# Class for Neural Network, functions for Data extraction and preparation:
class NN_Russian_digits:
 
    def __init__(self, d=[], l=[]):
        self.saver = saver
        self.weights = None
        self.d = d
        self.l = l
        self.out = None
        
    def dataset(self, test_files):
        self.d, self.l = [], []
        for test_file in test_files:
            signal, sampling_freq = librosa.load(test_file,sr=16000)
            with warnings.catch_warnings():
                    features_mfcc = mfcc(signal, 16000, nfft = 4000, numcep = 13)
                    pad_width = 13 - features_mfcc.shape[1]
                    features_mfcc = np.pad(features_mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
                    warnings.filterwarnings('ignore')
            warnings.filterwarnings("ignore")
            index = test_file.find('.') - 1
            original_label = test_file[index]
            original_label = int(original_label)
            self.l.append(original_label)
            self.d.append(features_mfcc)
        return self.d, self.l
    
    def label_append(self, y):
        z = []
        for i in range(len(y)):
            aa = [0]*10
            aa[y[i]] = y[i]
            z.append(aa)
        return z
    
    def feature_append(self, data_features):
        tt = []
        for i in range(len(data_features)):
            tt.append(np.shape(np.transpose(data_features[i])[1]))
        tt = np.unique(tt)
        u = 498
        for i in range(len(data_features)):
            data_features[i] = np.concatenate((data_features[i], np.zeros(((u-len(data_features[i])),13))))  
        return data_features
    
#     def suy_loss(self): 
#         def suy_loss(y_true, y_pred):
#             print('ok')
#             return K.mean(y_pred - y_true * K.log(y_pred + K.epsilon()), axis=-1)
#         return suy_loss
    
    def test_cosine_loss_4(self, w):
        def test_cosine_loss_4(y_true, y_pred, batch_size = 64):
            print(np.shape(y_true), np.shape(y_pred))
            print(np.shape(w), np.shape(self.out))
            var_weights = w
            normed_weights = tf.nn.l2_normalize(var_weights, (0,1), 1e-15, name='weights_norm')
            normed_features = tf.nn.l2_normalize(self.out, (0,1), 1e-15, name='features_norm')
            cosine = tf.matmul(normed_features, normed_weights)
            cosine = cosine - 0.25*y_true
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=25.*cosine))
            return K.sum(y_pred - y_pred, axis=-1) + loss
        print ("ok")
        return test_cosine_loss_4
    
    def ArcFaceLoss(self, w, scale=64., margin=0.25, easy_margin=True, scope=None):
        def ArcFaceLoss(y_true, y_pred, batch_size = 64):
            '''
            margin should in range [0, pi/2)
            '''
            print(np.shape(y_true), np.shape(y_pred))
            print(np.shape(w), np.shape(self.out))
            cos_m = math.cos(margin)
            sin_m = math.sin(margin)
            mm = math.sin(math.pi - margin) * margin
            threshold = math.cos(math.pi - margin)

            var_weights = w
            normed_weights = tf.nn.l2_normalize(var_weights, (0,1), 1e-15, name='weights_norm')
            normed_features = tf.nn.l2_normalize(self.out, (0,1), 1e-15, name='features_norm')
            cosine = tf.matmul(normed_features, normed_weights)
            cosine_theta_2 = tf.pow(cosine, 2., name='cosine_theta_2')

            sine_theta = tf.pow(1. - cosine_theta_2, .5, name='sine_theta')
    
            cosine_theta_m = scale * (cos_m * cosine - sin_m * sine_theta)

            if easy_margin:
                clip_mask = tf.to_float(cosine >= 0.) * scale * cosine * y_true
            else:
                clip_mask = tf.to_float(cosine >= threshold) * scale * mm * y_true

            cosine = scale * cosine * (1. - y_true) + tf.where(clip_mask > 0., cosine_theta_m, clip_mask)

            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,
                                                                                logits=cosine), name='arc_loss')
            print('ok')
            return K.sum(y_pred - y_pred, axis=-1) + loss
        return ArcFaceLoss


    def train_data(self, df, y):
        xtrain, xtest, ytrain, ytest = train_test_split(df, y, test_size=0.15, random_state=96)
        model = Sequential()
        model.add(LSTM(128, input_shape=(np.shape(df[0])), return_sequences=True))
        model.add(Flatten())
        model.add(Dense(64, activation='selu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='selu'))
        model.add(Dropout(0.5))
        model.add(Dense(10))
        self.weights = model.get_weights()
        # specify the layer index for weights and feature output:
        self.weights = self.weights[7]
        self.out = model.layers[4].output
        model.add(Activation('softmax'))
        # specify loss to be used:
        model.compile(optimizer='sgd',
                      loss=NN_Russian_digits.ArcFaceLoss(self, self.weights),
                      metrics=['accuracy'])
        return NN_Russian_digits.fit_data(self, xtrain, xtest, ytrain, ytest, model)
    
    
    #     # function for generating generators for data prior to fitting:
#     def generator(self, x, y, batch_size):
#         curIndex = 0
#         batch_x = np.zeros((batch_size,np.shape(x)[1],np.shape(x)[2]))
#         batch_y = np.zeros((batch_size,np.shape(y)[1]))
#         while True:
#             for i in range(batch_size):            
#                 batch_x[i] = x[curIndex,:,:]
#                 batch_y[i] = y[curIndex,:]
#                 i += 1;
#                 if i == np.shape(x)[0]:
#                     i = 0
#             yield batch_x, batch_y

        
    def fit_data(self, xtrain, xtest, ytrain, ytest, model):
        # use model.fit generator if we are dealing with generators:
        model.fit(x = np.asarray(xtrain), y = np.array(ytrain), validation_data=(np.asarray(xtest), np.array(ytest)), 
                  batch_size=64, epochs=100, verbose=2)
        model.save('NN_sp_Russian_digits2.h5') 
        model.save_weights('NN_sp_digit_weights.hdf5')
        return model
        
    def evaluate(self, xt, yt, model):
        metrics = model.evaluate(np.asarray(xt), np.array(yt))
        print('training data results: ')
        t = []
        for i in range(len(model.metrics_names)):
            t.append(str(model.metrics_names[i]) + ": " + str(metrics[i]))
        return t

    # prepare data from input folder:
    def prepare_data(self, input_folder):
        test_files = []
        for root,dirs,files in os.walk(input_folder):
            for filename in (x for x in files if '.wav' in x):
                filepath = os.path.join(root, filename)
                filepath = filepath.replace("\\","/")
                test_files.append(filepath)
            xt,yt = NN_Russian_digits.dataset(self, test_files)
            yt = NN_Russian_digits.label_append(self, yt)
            yt = np.int32(make_true(yt))
            xt = NN_Russian_digits.feature_append(self, xt)
        # get audio mfcc features from directory and return:
        return np.asarray(xt), np.asarray(yt)

In [107]:
# provide input directory path (for audio only):
input_folder = "C:/Users/akilesh/Desktop/Nsu big data books/train1"

In [92]:
# mnist = tf.keras.datasets.mnist

In [93]:
saver = tf.train.Saver

In [195]:
s = NN_Russian_digits()

In [16]:
# take mnist data set:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [17]:
x_train = x_train.reshape(np.shape(x_train)[0],28,28,1)
x_test = x_test.reshape(np.shape(x_test)[0],28,28,1)

In [66]:
df, y = s.prepare_data(input_folder)    

In [97]:
df = df.reshape(np.shape(df)[0],np.shape(df)[1],np.shape(df)[2])

In [98]:
df.shape

(1776, 498, 13)

In [18]:
# We need to provide one-hot labels as the input and for training:
label_one_hot = tf.one_hot(y, 10, on_value=1., off_value=0., axis=-1, dtype=tf.float32)
sess.run(label_one_hot)
label_one_hot = label_one_hot.eval(session=sess) 

In [19]:
label_one_hot_test = tf.one_hot(y_test, 10, on_value=1., off_value=0., axis=-1, dtype=tf.float32)
sess.run(label_one_hot_test)
label_one_hot_test = label_one_hot_test.eval(session=sess) 

In [214]:
# label_one_hot_one = label_one_hot.reshape(np.shape(label_one_hot)[0],np.shape(label_one_hot)[1],1)

In [18]:
label_one_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [188]:
S_New = np.asarray(S_New)
# S_New = S_New.reshape(np.shape(S_New)[0],np.shape(S_New)[1],np.shape(S_New)[2],1)
S_New_l = np.asarray(S_New_l)

In [189]:
xtrain, xtest, ytrain, ytest = train_test_split(S_New, S_New_l, test_size=0.20, random_state=96)

In [46]:
len(np.shape(df[0]))

2

In [ ]:
model2 = s.train_data(xtrain, ytrain)

(?, ?) (?, 10)
(64, 10) (?, 64)
ok
Train on 6113880 samples, validate on 1078920 samples
Epoch 1/100
 - 2658s - loss: 20.9864 - acc: 0.7807 - val_loss: 20.8636 - val_acc: 0.8431
Epoch 2/100
 - 2687s - loss: 20.9904 - acc: 0.7467 - val_loss: 20.9324 - val_acc: 0.8318
Epoch 3/100
 - 2679s - loss: 21.0354 - acc: 0.7369 - val_loss: 20.9639 - val_acc: 0.8283
Epoch 4/100
 - 2643s - loss: 21.0678 - acc: 0.7321 - val_loss: 20.9903 - val_acc: 0.8375
Epoch 5/100
 - 2632s - loss: 21.0933 - acc: 0.7290 - val_loss: 21.0262 - val_acc: 0.8457
Epoch 6/100
 - 2614s - loss: 21.1134 - acc: 0.7279 - val_loss: 21.0498 - val_acc: 0.8284
Epoch 7/100
 - 2612s - loss: 21.1311 - acc: 0.7280 - val_loss: 21.0555 - val_acc: 0.8182
Epoch 8/100
 - 2606s - loss: 21.1544 - acc: 0.7281 - val_loss: 21.0892 - val_acc: 0.8388
Epoch 9/100
 - 2618s - loss: 21.1684 - acc: 0.7279 - val_loss: 21.0956 - val_acc: 0.8434
Epoch 10/100
 - 2605s - loss: 21.1816 - acc: 0.7280 - val_loss: 21.1071 - val_acc: 0.8160
Epoch 11/100
 - 2615

In [188]:
# tf.flags.DEFINE_integer("batch_size", 64, "Batch size during training")
# tf.flags.DEFINE_integer("eval_batch_size", 8, "Batch size during evaluation")

In [311]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 10, 10, 64)        128       
_________________________________________________________________
flatten_18 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_43 (Dense)             (None, 64)                409664    
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 10)                650       
__________

In [24]:
# xtrain, xtest, ytrain, ytest = train_test_split(df, label_one_hot, test_size=0.90, random_state=96)

In [144]:
# works only when you put batch size equal to size of test set:
model2.evaluate(xtest,ytest,batch_size=840)

356/356 [==============================] - 9s 24ms/step


[16.170528411865234, 0.7415730357170105]

In [145]:
sss = model2.predict(df)

In [146]:
sss.shape

(1776, 10)

In [147]:
sss = sss[:1000]

In [27]:
# recieve feature output from layer before the last dense layer, provide the layer name from the model summary above:
layer_name = 'dense_2'
intermediate_layer_model = Model(input = model2.input, output = model2.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(x_test, batch_size=64)

/home/sivaswamy/tf/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
model2.weights

[<tf.Variable 'lstm_2/kernel:0' shape=(13, 1024) dtype=float32_ref>,
 <tf.Variable 'lstm_2/recurrent_kernel:0' shape=(256, 1024) dtype=float32_ref>,
 <tf.Variable 'lstm_2/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dense_4/kernel:0' shape=(127488, 64) dtype=float32_ref>,
 <tf.Variable 'dense_4/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_5/kernel:0' shape=(64, 64) dtype=float32_ref>,
 <tf.Variable 'dense_5/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_6/kernel:0' shape=(64, 10) dtype=float32_ref>,
 <tf.Variable 'dense_6/bias:0' shape=(10,) dtype=float32_ref>]

In [32]:
# get weights of the last layer
w = model2.weights[6]

In [156]:
# for future fitting:
# model2.fit(X,Y)

In [130]:
from keras.models import load_model

# model2 = load_model('NN_Russian_digits2.h5', 
#                     custom_objects={'test_cosine_loss_3': test_cosine_loss_3(w,intermediate_output)})
model2 = load_model('NN_Russian_digits2.h5', 
                    custom_objects={'test_cosine_loss_4': s.test_cosine_loss_4(w)})

ok
(?, ?) (?, 10)


In [19]:
x_train[0]

array([[[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
      

In [19]:
visible = Input(shape=(np.shape(df[0])[0],  np.shape(df[0])[1], 1))
conv1 = Conv2D(1, kernel_size=(1,1), activation='selu')(visible)
flat = Flatten()(conv1)
d1 = Dense(10)(flat)
model = Model(inputs=visible, outputs=d1)

In [20]:
j = tf.convert_to_tensor(np.float32(df[:1000]))

In [21]:
j

<tf.Tensor 'Const:0' shape=(1000, 498, 13, 1) dtype=float32>

In [22]:
sss = model(j)

In [23]:
sss

<tf.Tensor 'model_1/dense_1/BiasAdd:0' shape=(?, 10) dtype=float32>

In [24]:
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [25]:
sess.run(sss)
sss = sss.eval(session=sess) 

In [26]:
sss.shape

(1000, 10)

In [148]:
yyy = label_one_hot[:1000]

In [149]:
new_sss = []
new_yyy = []
for i in range(0, len(sss)):
    new_sss.append(np.matmul(np.transpose(sss[i][np.newaxis]), sss[i][np.newaxis]))
#     new_yyy.append(np.matmul(np.transpose(yyy[i][np.newaxis]), yyy[i][np.newaxis]))

In [150]:
np.shape(new_sss)

(1000, 10, 10)

In [48]:
new_yyy[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [180]:
# lamba wave:
lambda_sss, lambda_yyy, without_lambda_yyy, cc = [], [], [], []
for i in range(0,len(new_sss)):
    f = 1
    for j in range (0, 9):
        k = f/10 
        lambda_sss.append(new_sss[i]*np.sqrt(k))
        lambda_yyy.append(yyy[i]*np.sqrt(k))
        without_lambda_yyy.append(yyy[i]*k)
        cc.append(i)
        f = f+1

In [181]:
true_l = []
for i in range(0, len(lambda_yyy)):
    true_l.append(np.max(without_lambda_yyy[i]))

In [182]:
cc

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 24,
 24,


In [183]:
S_New, S_New_l = [], []
for i in range(0, len(true_l)):
    c = np.round(1 - true_l[i], 1)
#     print (c, single_l[i])
    for j in range(0, len(true_l)):
        if true_l[i] + true_l[j] == 1.:
            if cc[i] != cc[j]:
#             print(lambda_yyy[i], lambda_yyy[j])
                S_New.append(lambda_sss[i]+lambda_sss[j]+(np.sqrt(true_l[i]*(1-true_l[i]**2)))*
                             np.matmul(lambda_sss[i],lambda_sss[j])+np.matmul(lambda_sss[j],lambda_sss[i]))
                S_New_l.append(lambda_yyy[i]+lambda_yyy[j])

In [186]:
(S_New_l)

[array([0.31622776, 0.9486833 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       dtype=float32),
 array([0.31622776, 0.        , 0.9486833 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       dtype=float32),
 array([0.31622776, 0.        , 0.        , 0.9486833 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       dtype=float32),
 array([0.31622776, 0.        , 0.        , 0.        , 0.9486833 ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       dtype=float32),
 array([0.31622776, 0.        , 0.        , 0.        , 0.        ,
        0.9486833 , 0.        , 0.        , 0.        , 0.        ],
       dtype=float32),
 array([0.31622776, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.9486833 , 0.        , 0.        , 0.        ],
       dtype=float32),
 array([0.31622776, 0.        , 0.      

In [185]:
true_l

[0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5

In [57]:
S_New_l[10000]

array([0.4472136, 0.8944272, 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       ], dtype=float32)

In [39]:
2*np.sqrt(0.5)

1.4142135623730951